In [1]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from bs4 import BeautifulSoup
import albumentations as A
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
import albumentations
import albumentations.pytorch
from matplotlib import pyplot as plt
import os
import random
import utility
import rasterio
import wandb
import tqdm
import shutil
import pandas as pd
import joblib

In [ ]:
import matplotlib
matplotlib.use('TkAgg')

In [2]:
# same seed
torch.manual_seed(0)

In [3]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [4]:
# CUDA
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
batch_size = 1
threshold = 0.5

### Dataset Stats

In [6]:
def values_not_in_array(array1, array2):
    """"function that takes two arrays as input and returns a list 
    containing all the values in the first array that are not found in
     the second array:"""
    return [value for value in array1 if value not in array2]

In [7]:
def read_file(filename):
    lines = []
    with open(filename, 'r') as f:
        for line in f:
            lines.append(line.strip())
    return lines

In [8]:
clf = joblib.load("inshore_offshore_clf_normal_model.pkl")

def inshore_offshore_classifier(img):

    """
    Takes in an image and classifies it as either offshore(1) or inshore(0)
    """

    
    img_vals = np.copy(img)
    img_50 = np.percentile(img_vals,50)
    img_80 = np.percentile(img_vals,80)
    img_90 = np.percentile(img_vals,90)
    img_30 = np.percentile(img_vals,30)
    
    features = np.array([[img_50, img_80, img_90, img_30]])
    return clf.predict(features)[0]

In [9]:
# file path creations
# annotations for all ships
allShipsTXT = os.listdir("annotations_yolo/")
allShips = [ship[:-4] for ship in allShipsTXT]
allImageJPG = os.listdir("images/")
allImage = [img[:-4] for img in allImageJPG]
nonShips = values_not_in_array(allImage,allShips)

In [32]:
len(nonShips), len(allShips)

(7141, 1859)

In [10]:
temp = allShips[:5]
temp

['15_14_26', '11_2_6', '06_9_5', '15_19_13', '03_10_21']

In [11]:
# array of all test annotation with ships 
allShipsTest = os.listdir("test_yolo/")
allShipsTest = [ship[:-4] for ship in allShipsTest]

In [12]:
len(allShipsTest)

736

In [13]:
# array of all inshore annotations
allInshore = read_file("main/test_inshore.txt")

In [14]:
# array of all offshore annotations
allOffshore = read_file("main/test_offshore.txt")

In [15]:
set1 = set(allShipsTest)
set2 = set(allInshore)
set3 = set(allOffshore)

inshoreShipTest = list(set1.intersection(set2))
offshoreShipTest = list(set1.intersection(set3))

In [16]:
len(inshoreShipTest), len(offshoreShipTest)

(117, 619)

In [17]:
# need to calc inshore ships, offshore ships, inshore no ships, and offshore ships

In [18]:
allShipsTXT = os.listdir("annotations_yolo/")
allShips = [ship[:-4] for ship in allShipsTXT]

In [25]:
# array of all inshore annotations
allInshoreTest = read_file("main/test_inshore.txt")
# array of all offshore annotations
allOffshoreTest = read_file("main/test_offshore.txt")
len(allInshoreTest), len(allOffshoreTest)

(766, 2234)

In [20]:
allTrain = read_file("main/train.txt")
len(allTrain)

6000

In [21]:
trainInshore = []
trainOffshore = []

In [22]:
for fp in allTrain:
    image_fp = "images/{0}.jpg".format(fp)
    #print(image_fp)
    img = np.array(Image.open(image_fp))
    
    img = img/255.0
    #print(img)
    offshore = inshore_offshore_classifier(img) == 1
    #print(offshore)
    if offshore:
        trainOffshore.append(fp)
    else:
        trainInshore.append(fp)
    

/home/alex/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/alex/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `k

In [23]:
len(trainOffshore), len(trainInshore)

(4222, 1778)

In [26]:
allInshore = allInshoreTest + trainInshore
len(allInshore)

2544

In [27]:
allOffshore = trainOffshore + allOffshoreTest
len(allOffshore)

6456

In [33]:
allInshoreSet = set(allInshore)
allOffshoreSet = set(allOffshore)
allShipsSet = set(allShips)
nonShipsSet = set(nonShips)

In [34]:
# Inshore, Ship
inShip = list(allInshoreSet.intersection(allShipsSet))
len(inShip)

382

In [35]:
# offshore, Ship
offShip = list(allOffshoreSet.intersection(allShipsSet))
len(offShip)

1477

In [36]:
# Inshore, no Ship
inNoShip = list(allInshoreSet.intersection(nonShipsSet))
len(inNoShip)

2162

In [37]:
# offshore, no Ship
offNoShip = list(allOffshoreSet.intersection(nonShipsSet))
len(offNoShip)

4979

### Test Set Data

In [4]:
def count_objects_in_annotations(folder_path, prefix):
    """
    Counts the total number of objects in YOLO-format annotations in a folder
    whose file names start with a given prefix.

    Args:
    - folder_path (str): the path to the folder containing the annotations
    - prefix (int or str): the prefix to match the first two characters of the file names

    Returns:
    - count (int): the total number of objects in the annotations
    """
    count = 0
    
    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file name starts with the given prefix
        if file_name.startswith(str(prefix)):
            # Load the annotation file
            with open(os.path.join(folder_path, file_name), 'r') as f:
                lines = f.readlines()

            # Count the number of objects in the annotation file
            count += len(lines)

    return count

In [7]:
# number of ships image 11
count_objects_in_annotations("annotations_yolo/","11")

746

In [8]:
# number of ships image 12
count_objects_in_annotations("annotations_yolo/","12")

760

In [9]:
# number of ships image 13
count_objects_in_annotations("annotations_yolo/","13")

273

In [10]:
# number of ships image 14
count_objects_in_annotations("annotations_yolo/","14")

351

In [11]:
# number of ships image 15
count_objects_in_annotations("annotations_yolo/","15")

248

### Loading Data

In [ ]:
class ShipDataset:
    def __init__(self, path, transform=None):
        self.path = path
        if path == "no_ship":
            self.files = nonShips
        elif path == "test":
            self.files = allShipsTest
        elif path == "in":
            self.files = inshoreShipTest
        elif path == "off":
            self.files = offshoreShipTest
        else:
            self.files = list(sorted(os.listdir(path)))
        
        self.transform = transform
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        if self.path in ["no_ship","test","in","off"]:
            file_image = self.files[idx] + '.jpg'
            file_label = self.files[idx] + '.xml'
        else:
            file_image = self.files[idx][:-3] + 'jpg'
            file_label = self.files[idx][:-3] + 'xml'

        img_path = os.path.join("images/", file_image)
        label_path = os.path.join("annotations/", file_label)
        #print(img_path,label_path)
        
        # Read an image with OpenCV, gray scale
        image = cv2.imread(img_path,0)
        
        image = image/255.0
        target = utility.generate_target(label_path)

        
        if self.transform:
            transformed = self.transform(image = image, bboxes = target['boxes'], labels = target['labels'])
            image = torch.Tensor(transformed['image'])
            target = {'boxes':torch.Tensor(transformed['bboxes']).reshape(-1,4), 'labels':torch.Tensor(transformed['labels'])}
        else:
            image = torch.tensor(image,dtype=torch.float32)
        
        image = torch.unsqueeze(image, dim=0)
            
        return image, target 

In [ ]:
class ShipDataset_YOLO:
    def __init__(self, path, transform=None):
        self.path = path
        if path == "no_ship":
            self.files = nonShips
        elif path == "test":
            self.files = allShipsTest
        elif path == "in":
            self.files = inshoreShipTest
        elif path == "off":
            self.files = offshoreShipTest
        else:
            self.files = list(sorted(os.listdir(path)))
        
        self.transform = transform
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        if self.path in ["no_ship","test","in","off"]:
            file_image = self.files[idx] + '.jpg'
            file_label = self.files[idx] + '.xml'
        else:
            file_image = self.files[idx][:-3] + 'jpg'
            file_label = self.files[idx][:-3] + 'xml'

        img_path = os.path.join("images/", file_image)
        label_path = os.path.join("annotations/", file_label)
        #print(img_path,label_path)
        
        # Read an image with OpenCV, gray scale
        image = cv2.imread(img_path,0)
        
        image = image#/255.0
        target = utility.generate_target(label_path)

        
        if self.transform:
            transformed = self.transform(image = image, bboxes = target['boxes'], labels = target['labels'])
            image = torch.Tensor(transformed['image'])
            target = {'boxes':torch.Tensor(transformed['bboxes']).reshape(-1,4), 'labels':torch.Tensor(transformed['labels'])}
        else:
            image = torch.tensor(image,dtype=torch.float32)
        
        image = torch.unsqueeze(image, dim=0)
            
        return image, target 

#### Faster R-CNN and RetinaNet Datasets and Loaders

In [ ]:
val_dataset = ShipDataset(
    path = 'test_yolo/',
    #transform=transform
)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,collate_fn=utility.collate_fn)
len(val_dataset)

In [ ]:
inshore_dataset = ShipDataset(
    path = 'in',
    #transform=transform
)

inshore_loader = torch.utils.data.DataLoader(inshore_dataset, batch_size=batch_size,collate_fn=utility.collate_fn)
len(inshore_dataset)

In [ ]:
offshore_dataset = ShipDataset(
    path = 'off',
    #transform=transform
)

offshore_loader = torch.utils.data.DataLoader(offshore_dataset, batch_size=batch_size,collate_fn=utility.collate_fn)
len(offshore_dataset)

#### YOLO Data

In [ ]:
val_dataset_yolo = ShipDataset_YOLO(
    path = 'test_yolo/',
    #transform=transform
)

val_loader_yolo = torch.utils.data.DataLoader(val_dataset_yolo, batch_size=batch_size,collate_fn=utility.collate_fn)
len(val_loader_yolo)

In [ ]:
inshore_dataset_yolo = ShipDataset_YOLO(
    path = 'in',
    #transform=transform
)

inshore_loader_yolo = torch.utils.data.DataLoader(inshore_dataset_yolo, batch_size=batch_size,collate_fn=utility.collate_fn)
len(inshore_dataset_yolo)

In [ ]:
offshore_dataset_yolo = ShipDataset_YOLO(
    path = 'off',
    #transform=transform
)

offshore_loader_yolo = torch.utils.data.DataLoader(offshore_dataset_yolo, batch_size=batch_size,collate_fn=utility.collate_fn)
len(offshore_dataset_yolo)

### Load RetinaNet

In [ ]:
model = torchvision.models.detection.retinanet_resnet50_fpn(num_classes=2)
model.load_state_dict(torch.load("models/retina300R2.pt"))
model.eval()
model.to(device)

### Load Faster R-CNN

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
num_classes = 2  # 1 class (ship) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
model.load_state_dict(torch.load("models/faster300ep.pt"))
model.eval()
model.to(device)

### Load YOLOv7 

In [ ]:
path = "models/yolov7d6.pt"
model = torch.hub.load("WongKinYiu/yolov7","custom",f"{path}",trust_repo=True)
model.eval()
model.to(device)

### Visualizing results

In [ ]:
def plot_ground_truth_and_predictions(model, data_loader, device,threshold, n,save_path):
    """
    For RetinaNet and Faster R-CNN
    model: model to predict
    data_loader: test_data loader to use
    device: the device to use
    threshold: confidence value to threshold a prediction
    n: random into scroll on images
    save_path: save path of images, must end in .jpg

    returns: none
    prints a random image from data_loader with both its ground truth and the models predictions
    """
    # Get a batch of validation data
    data_loader = iter(data_loader)
    #images, targets = next(data_loader)
    images, targets = next(data_loader)
    
    for i in range(n):
        images, targets = next(data_loader)

    images = images[0]
    # Send the images and targets to the device
    images = images.to(device)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    images = torch.unsqueeze(images, dim=0)
    # Run the model on the images
    # TODO: ADD THRESHOLDING TO MODEL
    # TODO: Add a boolean for yolo model 
    outputs = model(images)


    # Loop over the images in the batch
    for i in range(len(images)):
        # Get the image, target boxes, and predicted boxes for this image
        print(images[i].shape)
        image = images[i].cpu().permute(1, 2, 0).numpy()
        print(image.shape)
        target_boxes = targets[i]['boxes'].cpu().numpy()

        # grabs all ids that are above the threshold value
        idx_list = []
        for idx, score in enumerate(outputs[0]['scores']) :
            if score > threshold : #select idx which meets the threshold
                idx_list.append(idx)

        predicted_boxes = outputs[i]['boxes'][idx_list].detach().cpu().numpy()

        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        # Plot the ground truth boxes on the first subplot
        axs[0].imshow(image,cmap="gray")
        for box in target_boxes:
            rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                     linewidth=2, edgecolor='g', facecolor='none')
            axs[0].add_patch(rect)
        axs[0].set_title('Ground Truth')
        axs[0].set_xticks([])
        axs[0].set_yticks([])

        # Plot the predicted boxes on the second subplot
        axs[1].imshow(image,cmap="gray")
        for box in predicted_boxes:
            rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                     linewidth=2, edgecolor='r', facecolor='none')
            axs[1].add_patch(rect)
        axs[1].set_title('Predictions')
        axs[1].set_xticks([])
        axs[1].set_yticks([])
        # Show the plot
        if save_path:
            plt.savefig(save_path,quality=100,dpi=500)
        plt.show()

In [ ]:
def plot_ground_truth_and_predictions_YOLO(model, data_loader, device,threshold, n,save_path):
    """
    For Yolo Model only!
    model: model to predict
    data_loader: test_data loader to use
    device: the device to use
    threshold: confidence value to threshold a prediction
    n: random into scroll on images
    save_path: save path of images, must end in .jpg

    returns: none
    prints a random image from data_loader with both its ground truth and the models predictions
    """
    # TODO: Add a boolean for yolo model 
    # Get a batch of validation data

    data_loader = iter(data_loader)
    #images, targets = next(data_loader)
    images, targets = next(data_loader)
    
    for i in range(n):
        images, targets = next(data_loader)

    # image on cpu for yolo
    images = torch.squeeze(images[0],0).cpu().numpy()

    # images = images[0]
    # # Send the images and targets to the device
    # #images = images.to(device)
    # targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    # images = torch.unsqueeze(images, dim=0)
    # Run the model on the images
    # TODO: ADD THRESHOLDING TO MODEL
    # TODO: Add a boolean for yolo model 
    outputs = model(images)

    # YOLO conversion
    outputs = outputs.pred[0]

    # Create a dictionary with keys 'boxes', 'labels', and 'scores'
    outputs = {
        'boxes': outputs[:, :4],  # only take the first four columns of the tensor
        'labels': outputs[:, 5].long()+1,
        'scores': outputs[:, 4]
    }
    outputs = [outputs]

    # Loop over the images in the batch
    for i in range(len(images)):
        # Get the image, target boxes, and predicted boxes for this image
        # make image 800, 800, 1
        image = images[ :, :, np.newaxis]
        target_boxes = targets[i]['boxes'].cpu().numpy()

        # grabs all ids that are above the threshold value
        idx_list = []
        for idx, score in enumerate(outputs[0]['scores']) :
            if score > threshold : #select idx which meets the threshold
                idx_list.append(idx)

        predicted_boxes = outputs[i]['boxes'][idx_list].detach().cpu().numpy()

        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        # Plot the ground truth boxes on the first subplot
        axs[0].imshow(image,cmap="gray")
        for box in target_boxes:
            rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                     linewidth=2, edgecolor='g', facecolor='none')
            axs[0].add_patch(rect)
        axs[0].set_title('Ground Truth')
        axs[0].set_xticks([])
        axs[0].set_yticks([])

        # Plot the predicted boxes on the second subplot
        axs[1].imshow(image,cmap="gray")
        for box in predicted_boxes:
            rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                     linewidth=2, edgecolor='r', facecolor='none')
            axs[1].add_patch(rect)
        axs[1].set_title('Predictions')
        axs[1].set_xticks([])
        axs[1].set_yticks([])
        # Show the plot
        
        if save_path:
            plt.savefig(save_path,quality=100,dpi=500)
        plt.show()

In [ ]:
# reg
#plot_ground_truth_and_predictions(model,noShip_loader,device,0.1,1)

In [ ]:
plot_ground_truth_and_predictions_YOLO(model,val_loader_yolo,device,0.1,1,False)

In [ ]:
for i in range(4):
    #plot_ground_truth_and_predictions(model,val_loader,device,i)

### Faster R-CNN and RetinaNet Metrics

In [ ]:
from tqdm import tqdm

labels = []
preds_adj_all = []
annot_all = []

for im, annot in tqdm(val_loader, position = 0, leave = True):
    im = list(img.to(device) for img in im)
    #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

    for t in annot:
        labels += t['labels']

    with torch.no_grad():
        preds_adj = utility.make_prediction(model, im, 0.5)
        preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
        preds_adj_all.append(preds_adj)
        annot_all.append(annot)

In [ ]:
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    sample_metrics += utility.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))] 
precision, recall, AP, f1, ap_class = utility.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
mAP = torch.mean(torch.tensor(AP,dtype=torch.float64))

In [ ]:
"Precision: {0} \n Recall: {1} \\n AP: {2} \n F1: {3} \n mAP {4}".format(precision.item(), recall.item(), AP.item(), f1.item(),mAP.item())

In [ ]:
def get_metric(model,datalaoder,threshold):
    """
    Not to be used with YOLO
    """

    t1 = time.time()
    labels = []
    preds_adj_all = []
    annot_all = []

    for im, annot in datalaoder:
        im = list(img.to(device) for img in im)
        #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

        for t in annot:
            labels += t['labels']

        with torch.no_grad():
            preds_adj = utility.make_prediction(model, im, threshold)
            preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
            preds_adj_all.append(preds_adj)
            annot_all.append(annot)
            break
    
    sample_metrics = []
    for batch_i in range(len(preds_adj_all)):
        sample_metrics += utility.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 
    
    true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))] 
    precision, recall, AP, f1, ap_class = utility.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
    mAP = torch.mean(torch.tensor(AP,dtype=torch.float64))
    t2 = time.time()
    t3 = t2-t1
    result = "Precision: {0:.3f} \n Recall: {1:.3f} \n AP: {2:.3f} \n F1: {3:.3f} \n mAP {4:.3f} \n time: {5:.3f} \n threshold: {6:}".format(precision.item(), recall.item(), AP.item(), f1.item(),mAP.item(), t3,threshold)
    print(result)
    return [precision.item(), recall.item(), AP.item(), f1.item(),mAP.item(), t3, threshold]

In [ ]:
tempred1 = get_metric(model,inshore_loader,0.2)
tempred1

### Test on no Ship images

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
 
# Read Images
img = mpimg.imread("images/01_1_1.jpg")
 
# Output Images
plt.imshow(img)

In [ ]:
allShipsTXT = os.listdir("annotations_yolo/")
allShips = [ship[:-4] for ship in allShipsTXT]

allImageJPG = os.listdir("images/")
allImage = [img[:-4] for img in allImageJPG]


In [ ]:
def values_not_in_array(array1, array2):
    """"function that takes two arrays as input and returns a list 
    containing all the values in the first array that are not found in
     the second array:"""
    return [value for value in array1 if value not in array2]

In [ ]:
# array of images file names that do not have ships
nonShips = values_not_in_array(allImage,allShips)
nonShips.sort()

In [ ]:
len(nonShips)

In [ ]:
noShips = []
for i in range(30):
    temp1 = "images/01_1_{0}.jpg".format((i+1))
    if i < 21:
        temp2 = "images/01_2_{0}.jpg".format((i+1))
        noShips.append(temp2)
    noShips.append(temp1)
    

In [ ]:
def test_no_ship(model, threshold, images):
    allIm = []
    for im in images:
        im = "images/{0}.jpg".format(im)
        image = cv2.imread(im,0)
        image = image/255.0
        image = torch.tensor(image,dtype=torch.float32)
        image = torch.unsqueeze(image, dim=0)
        image = image.to(device)
        allIm.append(image)
    
    
    numWrong = 0
    numIm = len(allIm)
    for im in allIm:
        im = torch.unsqueeze(im, dim=0)
        pred = model(im)

        if torch.sum(pred[0]["scores"]>threshold).item() > 0:
            #print(im)
            numWrong += 1

    return (numIm-numWrong)/float(numIm)


In [ ]:
test_no_ship(model,threshold, nonShips)

### Working with YOLOv7

In [ ]:
image1 = cv2.imread("gee/2020-01-05_5.jpg",0)

image1 = np.clip(image1, -20, 0)

# for some reason have to rescale or else when saving image it will be dark
image1 = (image1-image1.min())/(image1.max()-image1.min())
#plt.imshow(image1,cmap="gray")
#plt.show()
#image = image1/255.0
image = torch.tensor(image1,dtype=torch.float32)
image = torch.unsqueeze(image, dim=0)
print(image.shape)
image = image.to(device)
test = [image]

In [ ]:
pred = model(image)

In [ ]:
pred.pred[0]

In [ ]:
image.shape

In [ ]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : #select idx which meets the threshold
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]


    return preds


In [ ]:
fasterPred = make_prediction(model,test,0.5)
fasterPred

In [ ]:
image.shape

In [ ]:
def make_prediction_yolo(model, img, threshold):
    model.eval()
    preds = model(img)
    
    preds = preds.pred[0]

    # Create a dictionary with keys 'boxes', 'labels', and 'scores'
    preds = {
        'boxes': preds[:, :4],  # only take the first four columns of the tensor
        'labels': preds[:, 5].long()+1,
        'scores': preds[:, 4]
    }
    preds = [preds]
    
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : #select idx which meets the threshold
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]


    return preds

In [ ]:
def get_metric_YOLO(model, dataloader, threshold):
    t1 = time.time()
    labels = []
    preds_adj_all = []
    annot_all = []

    for im, annot in dataloader:
        
        for t in annot:
            
            labels += t['labels']
        
        with torch.no_grad():
            ### make a new function for yolo, need to move to cpu first
            # will slow down time significantly
            im = torch.squeeze(im[0],0).cpu().numpy()
            preds_adj = make_prediction_yolo(model, im, threshold)
            preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
            preds_adj_all.append(preds_adj)
            annot_all.append(annot)
    
    sample_metrics = []
    for batch_i in range(len(preds_adj_all)):
        sample_metrics += utility.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 
    true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))] 
    precision, recall, AP, f1, ap_class = utility.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
    mAP = torch.mean(torch.tensor(AP,dtype=torch.float64))
    t2 = time.time()
    t3 = t2-t1
    result = "Precision: {0:.3f} \n Recall: {1:.3f} \n AP: {2:.3f} \n F1: {3:.3f} \n mAP {4:.3f} \n time: {5:.3f} \n threshold: {6:}".format(precision.item(), recall.item(), AP.item(), f1.item(),mAP.item(), t3,threshold)
    print(result)
    return [precision.item(), recall.item(), AP.item(), f1.item(),mAP.item(), t3, threshold]
            

In [ ]:
tempYolopred = get_metric_YOLO(model, inshore_loader_yolo, 0.2)
tempYolopred

In [ ]:
t = list(tempYolopred)
t = [t]
t

In [ ]:
cols = ["Precision", "Recall", "AP" , "f1" ,"mAP", "Time (seconds)", "Threshold"]
df = pd.DataFrame(t, columns=cols)
df.head()

In [ ]:
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    sample_metrics += utility.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 
print(sample_metrics[0])
true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))] 
precision, recall, AP, f1, ap_class = utility.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
mAP = torch.mean(torch.tensor(AP,dtype=torch.float64))

In [ ]:
"Precision: {0} \n Recall: {1} \\n AP: {2} \n F1: {3} \n mAP {4}".format(precision.item(), recall.item(), AP.item(), f1.item(),mAP.item())

In [ ]:
import matplotlib
matplotlib.use('TkAgg')

In [ ]:
plt.imshow(image1)
plt.show()

In [ ]:
image1 = cv2.imread("gee/2020-01-05_5.jpg",0)
image = image1/255.0

In [ ]:
model(image1).pred[0]

In [ ]:
with rasterio.open(r'/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/gee_data/2020-01-05.tif') as src:
    img = src.read()
    img = np.clip(img, -20, 0)
    plt.imshow(img[0],cmap="gray")
    plt.show()
    img = (img-img.min())/(img.max()-img.min())
    plt.imshow(img[0],cmap="gray")
    
    plt.show()

In [ ]:
img.shape

In [ ]:
res = model(img)

In [ ]:
# Extract the bounding boxes, object confidence scores, and class IDs
boxes = res.xyxy[0]  # xyxy coordinates of boxes for first image in batch
scores = res.pred[0][:, 4]  # object confidence scores for first image in batch
class_ids = res.pred[0][:, 5]  # class IDs for first image in batch

In [ ]:
boxes, scores

In [ ]:
res.xyxy

In [ ]:
res.pandas().xyxy[0].to_numpy()

In [ ]:
res.pandas().xyxy[0].to_numpy()[0]

In [ ]:
prediction = model(test)
prediction

In [ ]:
sum(prediction[0]["scores"]>threshold).item()

In [ ]:
torch.sum(prediction[0]["scores"]>0.5).item()

In [ ]:
allFiles = os.listdir("images/")

In [ ]:
allFiles.sort()

In [ ]:
allFiles[:5]

In [ ]:
import joblib

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
from sklearn.neighbors import _dist_metrics

In [ ]:
clf = joblib.load("inshore_offshore_clf_model.pkl")
img = cv2.imread("gee/2020-01-06_5.jpg",0)
img_vals = np.copy(img)
img_50 = np.percentile(img_vals,50)
img_80 = np.percentile(img_vals,80)
img_90 = np.percentile(img_vals,90)
img_30 = np.percentile(img_vals,30)
    
features = np.array([[img_50, img_80, img_90, img_30]])
calssres = clf.predict(features)

In [ ]:
calssres[0]

## GEE Image Reading and Testing

In [ ]:
with rasterio.open(r'/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/gee_data/2020-01-11.tif') as src:
    img = src.read()
    plt.imshow(img[0])

In [ ]:
with rasterio.open(r'/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/gee_data/2020-01-06.tif') as src:
    img = src.read()
    img = np.clip(img, -20, 0)
    plt.imshow(img[0],cmap="gray")

In [ ]:
with rasterio.open(r'/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/gee_data/2020-01-05.tif') as src:
    img = src.read()[0]
    img = np.clip(img, -20, 0)
    plt.imshow(img,cmap="gray")
    # img = Image.fromarray(img)
    # img = img.convert("L")
    # img.save("test.jpg")

In [ ]:
## TODO: Fix YOLO
## TODO: Figure out thresholding
## TODO: Metrics for no ships


In [ ]:
## TODO: plot bounding box on large image

In [ ]:
def image_splitter(img_fp):
    """
    Takes the image filepath returns m x n array of the subimages
    pads with 0
    """
    #     img = gdal.Open(img_fp)
    #     img_array = img.GetRasterBand(1).ReadAsArray()
    # note have to add in rescaled helper fn

    # define splitting helper function
    # got this from https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7
    def reshape_split(image, kernel_size):
        img_height, img_width = image.shape
        tile_height, tile_width = kernel_size
        
        tiled_array = image.reshape(img_height // tile_height,
                                tile_height,
                                img_width // tile_width,
                                tile_width)
        tiled_array = tiled_array.swapaxes(1,2)
        return tiled_array


    # get image name to return
    # splits full fp by \, then gets the YYYY-MM-DD_#.jpg, then cuts off .jpg 
    img_name = img_fp.split('\\')[-1][:-4]



    with rasterio.open(img_fp) as src:
        img_array = src.read()[0]
        #print(img_array.shape)
        
    # have to clip values to take away gray tint from image
    # and have to do this so that inshore offshore classifier works
    img_array = np.clip(img_array, -20, 0)
    
    img_height, img_width = img_array.shape
    # get next biggest multiple of 800
    # TODO: Make it closest multiple of 800 instead OR PAD WITH BLACK
    new_height = img_height + (800 - img_height % 800)
    new_width = img_width + (800 - img_width % 800)
    
    # calculate number of pixels to pad
    delta_w = new_width - img_width
    delta_h = new_height - img_height
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    # pad image with 0
    image_pad = np.pad(img_array, ((top, bottom), (left, right)), mode='constant', constant_values=-20)
    
    # for some reason have to rescale or else when saving image it will be dark
    rescaled = 255*(image_pad-image_pad.min())/(image_pad.max()-image_pad.min())

    # normalize between 0 and 1
    # MAKE A BETTER RESCALING
    rescale_normalized = rescaled / 255
    
    # split image into subimages
        # will return array [m, n, 800, 800] where there are an m x n number of images with size 800x800 
    split = reshape_split(rescale_normalized, kernel_size=(800,800))
    
    return split, img_name

In [ ]:
def plot_large_image(model,image_path,threshold,save_path):
    """
    model: pytorch 
    image_path: path tif file
    threshold: confidence threshold for model
    save_path: path to save plot (must be jpg), if none will not save
    """

    # split image and save size for future
    split_image, _ = image_splitter(image_path)
    split_size = split_image.shape[:2]

    # flatten image and format to work with pytroch device
    flattened = np.reshape(split_image, (-1, split_image.shape[2], split_image.shape[3]))
    torch_split_img = np.array(flattened)
    torch_split_img = torch.tensor(torch_split_img,dtype=torch.float32)
    torch_split_img = torch.unsqueeze(torch_split_img, dim=0)
    torch_split_img = torch_split_img.permute(1,0,2,3)
    torch_split_img = torch_split_img.to(device)

    # predict bounding boxes
    pred = model(torch_split_img)
    # reshape to match image split
    pred = np.reshape(pred,split_size)


    # Create a 3x3 plot with a shared axis
    fig, axs = plt.subplots(split_size[0], split_size[1], sharex=True, sharey=True, figsize=(6, 6))

    # Loop over the images and annotations and plot them on the corresponding subplot
    for i in range(split_size[0]):
        for j in range(split_size[1]):
            # Plot the image
            axs[i, j].imshow(split_image[i, j], cmap='gray')
            
            # Remove the axis labels and ticks
            axs[i, j].set_xticks([])
            axs[i, j].set_yticks([])
            axs[i, j].axis('off')
            # Loop over the annotations and plot them as bounding boxes
            curPred = pred[i, j]
            for k in range(len(curPred["scores"])):
                # checks if above threshold
                if curPred["scores"][k] > threshold:
                    annotation = curPred['boxes'][k]
                    annotation = annotation.cpu().detach().numpy()
                    left, upper, right, lower = annotation
                    width, height = right - left, lower - upper
                    rect = plt.Rectangle((left, upper), width, height, linewidth=1, edgecolor='r', facecolor='none')
                    axs[i, j].add_patch(rect)

    # Remove the space between the images
    plt.subplots_adjust(wspace=0, hspace=0)
    if save_path:
        plt.savefig(save_path,quality=100,dpi=500)
    # Show the plot
    plt.show()

In [ ]:
#plot_large_image(model,r'/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/gee_data/2020-01-05.tif',threshold,None)

In [ ]:
faster_rcnn = model

In [ ]:
testing, _ = image_splitter('/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/gee_data/2020-01-05.tif')

In [ ]:
testing.shape[:2]

In [ ]:
flattened = np.reshape(testing, (-1, testing.shape[2], testing.shape[3]))

In [ ]:
inshoreImg = np.array(flattened)
inshoreImg = torch.tensor(inshoreImg,dtype=torch.float32)
inshoreImg = torch.unsqueeze(inshoreImg, dim=0)
inshoreImg = inshoreImg.permute(1,0,2,3)
inshoreImg = inshoreImg.to(device)

In [ ]:
testPred = faster_rcnn(inshoreImg)

In [ ]:
testPred = np.reshape(testPred,(3,3))

In [ ]:
testPred[0,0]

In [ ]:
# Create a 3x3 plot with a shared axis
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(6, 6))

# Loop over the images and annotations and plot them on the corresponding subplot
for i in range(3):
    for j in range(3):
        # Plot the image
        axs[i, j].imshow(testing[i, j], cmap='gray')
        
        # Remove the axis labels and ticks
        axs[i, j].set_xticks([])
        axs[i, j].set_yticks([])
        axs[i, j].axis('off')
        # Loop over the annotations and plot them as bounding boxes
        #print(testPred)
        curPred = testPred[i, j]
        #print(len(curPred["scores"]))
        for k in range(len(curPred["scores"])):
            #print(curPred["scores"])
            if curPred["scores"][k] > 0.5:
                annotation = curPred['boxes'][k]
                annotation = annotation.cpu().detach().numpy()
                left, upper, right, lower = annotation#.detach().numpy()
                width, height = right - left, lower - upper
                rect = plt.Rectangle((left, upper), width, height, linewidth=1, edgecolor='r', facecolor='none')
                axs[i, j].add_patch(rect)

# Set the axis labels
#fig.text(0.5, 0.04, 'X', ha='center')
#fig.text(0.04, 0.5, 'Y', va='center', rotation='vertical')

# Remove the space between the images
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('my_plot.jpg',quality=100,dpi=500)
# Show the plot
plt.show()

In [2]:
imagebig = cv2.imread("BigData/01.jpg",0)
imagebig.shape

(16000, 24000)

numpy.ndarray